In [86]:
from PIL import Image
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Bernoulli
from torch.utils.data import Sampler
import torchvision.datasets as datasets
from tqdm import tqdm
import plotly.express as px
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score

In [77]:
def get_transform():
    
    mean=[0.485,0.456,0.406]
    std=[0.229,0.224,0.225]
    normalize = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=mean,std=std)])
    size_transform = transforms.Resize([84,84])
    
    eval_transform = transforms.Compose([size_transform,normalize])
    return eval_transform
     
def image_loader(path):
    
    p = Image.open(path)
    p = p.convert('RGB')
    final_transform = get_transform()
    p = final_transform(p)
    return p

class CustomDataset_Support(object):
    def __init__(self, df):
        self.df = df
        
    def __getitem__(self, idx):
        dir_p = '../input/snap-retail-data/final_data/query_data/support/'
        img_p = dir_p+self.df.loc[idx, 'class'].astype(str) + '_' + self.df.loc[idx, 'image_id']
        target = self.df.loc[idx,'class_code']
        img= image_loader(img_p)
        
        return img, target
        
    def __len__(self):
        return len(self.df)
    
# Dataset class for Validation - 10%of data
class CustomDataset_Query(object):
    def __init__(self, df, is_trained_class=True):
        self.df = df
        self.is_trained_class = is_trained_class
        
    def __getitem__(self, idx):
        if self.is_trained_class:
            dir_p = '../input/snap-retail-data/final_data/query_data/valid_query/'
            img_p = dir_p+self.df.loc[idx, 'class'].astype(str) + '_' +self.df.loc[idx, 'image_id']
        else:
            dir_p = '../input/snap-retail-data/final_data/query_data/test_query/'
            img_p = dir_p+self.df.loc[idx, 'class'].astype(str) + '_' +self.df.loc[idx, 'image_id']
        target = self.df.loc[idx,'class_code']
        img= image_loader(img_p)
        
        return img, target
        
    def __len__(self):
        return len(self.df)

## Model

In [78]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class DropBlock(nn.Module):
    def __init__(self, block_size):
        super(DropBlock, self).__init__()

        self.block_size = block_size


    def forward(self, x, gamma):
        # shape: (bsize, channels, height, width)

        if self.training:
            batch_size, channels, height, width = x.shape
            
            bernoulli = Bernoulli(gamma)
            mask = bernoulli.sample((batch_size, channels, height - (self.block_size - 1), width - (self.block_size - 1))).cuda()
            block_mask = self._compute_block_mask(mask)
            countM = block_mask.size()[0] * block_mask.size()[1] * block_mask.size()[2] * block_mask.size()[3]
            count_ones = block_mask.sum()

            return block_mask * x * (countM / count_ones)
        else:
            return x

    def _compute_block_mask(self, mask):
        left_padding = int((self.block_size-1) / 2)
        right_padding = int(self.block_size / 2)
        
        batch_size, channels, height, width = mask.shape
        non_zero_idxs = mask.nonzero()
        nr_blocks = non_zero_idxs.shape[0]

        offsets = torch.stack(
            [
                torch.arange(self.block_size).view(-1, 1).expand(self.block_size, self.block_size).reshape(-1), # - left_padding,
                torch.arange(self.block_size).repeat(self.block_size), #- left_padding
            ]
        ).t().cuda()
        offsets = torch.cat((torch.zeros(self.block_size**2, 2).cuda().long(), offsets.long()), 1)
        
        if nr_blocks > 0:
            non_zero_idxs = non_zero_idxs.repeat(self.block_size ** 2, 1)
            offsets = offsets.repeat(nr_blocks, 1).view(-1, 4)
            offsets = offsets.long()

            block_idxs = non_zero_idxs + offsets
            padded_mask = F.pad(mask, (left_padding, right_padding, left_padding, right_padding))
            padded_mask[block_idxs[:, 0], block_idxs[:, 1], block_idxs[:, 2], block_idxs[:, 3]] = 1.
        else:
            padded_mask = F.pad(mask, (left_padding, right_padding, left_padding, right_padding))
            
        block_mask = 1 - padded_mask#[:height, :width]
        return block_mask
    

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, drop_rate=0.0, drop_block=False,
                 block_size=1,max_pool=True):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.LeakyReLU(0.1)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv3x3(planes, planes)
        self.bn3 = nn.BatchNorm2d(planes)
        self.maxpool = nn.MaxPool2d(stride)
        self.downsample = downsample
        self.stride = stride
        self.drop_rate = drop_rate
        self.num_batches_tracked = 0
        self.drop_block = drop_block
        self.block_size = block_size
        self.DropBlock = DropBlock(block_size=self.block_size)
        self.max_pool = max_pool

    def forward(self, x):
        self.num_batches_tracked += 1

        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        
        if self.max_pool:
            out = self.maxpool(out)

        if self.drop_rate > 0:
            if self.drop_block == True:
                feat_size = out.size()[2]
                keep_rate = max(1.0 - self.drop_rate / (20*2000) * (self.num_batches_tracked), 1.0 - self.drop_rate)
                gamma = (1 - keep_rate) / self.block_size**2 * feat_size**2 / (feat_size - self.block_size + 1)**2
                out = self.DropBlock(out, gamma=gamma)
            else:
                out = F.dropout(out, p=self.drop_rate, training=self.training, inplace=True)

        return out
    
class ResNet(nn.Module):

    def __init__(self, block, n_blocks, drop_rate=0.0, dropblock_size=5, max_pool=True):
        super(ResNet, self).__init__()

        self.inplanes = 3
        self.layer1 = self._make_layer(block, n_blocks[0], 64,
                                       stride=2, drop_rate=drop_rate)
        self.layer2 = self._make_layer(block, n_blocks[1], 160,
                                       stride=2, drop_rate=drop_rate)
        self.layer3 = self._make_layer(block, n_blocks[2], 320,
                                       stride=2, drop_rate=drop_rate, drop_block=True, block_size=dropblock_size)
        self.layer4 = self._make_layer(block, n_blocks[3], 640,
                                       stride=2, drop_rate=drop_rate, drop_block=True, block_size=dropblock_size,max_pool=max_pool)

        self.drop_rate = drop_rate
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


    def _make_layer(self, block, n_block, planes, stride=1, drop_rate=0.0, drop_block=False, block_size=1,max_pool=True):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        if n_block == 1:
            layer = block(self.inplanes, planes, stride, downsample, drop_rate, drop_block, block_size,max_pool=max_pool)
        else:
            layer = block(self.inplanes, planes, stride, downsample, drop_rate)
        layers.append(layer)
        self.inplanes = planes * block.expansion

        for i in range(1, n_block):
            if i == n_block - 1:
                layer = block(self.inplanes, planes, drop_rate=drop_rate, drop_block=drop_block,
                              block_size=block_size)
            else:
                layer = block(self.inplanes, planes, drop_rate=drop_rate)
            layers.append(layer)

        return nn.Sequential(*layers)

    def forward(self, x, is_feat=False):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        return x
    
    def resnet12(drop_rate=0.0, max_pool=True, **kwargs):
        """Constructs a ResNet-12 model.
        """
        model = ResNet(BasicBlock, [1, 1, 1, 1], drop_rate=drop_rate, max_pool=max_pool, **kwargs)
        return model

In [79]:
class FRN(nn.Module):
    
    def __init__(self,way=None,shots=None,is_pretraining=False,num_cat=None):
        
        super().__init__()

        num_channel = 640
        self.feature_extractor = ResNet.resnet12()

        self.shots = shots
        self.way = way

        # number of channels for the feature map, correspond to d in the paper
        self.d = num_channel
        
        # temperature scaling, correspond to gamma in the paper
        self.scale = nn.Parameter(torch.FloatTensor([1.0]),requires_grad=True)
        
        # H*W=5*5=25, resolution of feature map, correspond to r in the paper
        self.resolution = 25

        # correpond to [alpha, beta] in the paper
        # if is during pre-training, we fix them to 0
        self.r = nn.Parameter(torch.zeros(2),requires_grad=not is_pretraining)

        if is_pretraining:
            # number of categories during pre-training
            self.num_cat = num_cat
            # category matrix, correspond to matrix M of section 3.6 in the paper
            self.cat_mat = nn.Parameter(torch.randn(self.num_cat,self.resolution,self.d),requires_grad=True)   
    

    def get_feature_map(self,inp):

        batch_size = inp.size(0)
        feature_map = self.feature_extractor(inp)
        
        feature_map = feature_map/np.sqrt(640)
        
#         print('Feature Extracted Shape: ', feature_map.shape)
        return feature_map.view(batch_size,self.d,-1).permute(0,2,1).contiguous() # N,HW,C
    

    def get_recon_dist(self,query,support,alpha,beta,Woodbury=True):
    # query: way*query_shot*resolution, d
    # support: way, shot*resolution , d
    # Woodbury: whether to use the Woodbury Identity as the implementation or not

        # correspond to kr/d in the paper
        reg = support.size(1)/support.size(2)
        
        # correspond to lambda in the paper
        lam = reg*alpha.exp()+1e-6

        # correspond to gamma in the paper
        rho = beta.exp()

        st = support.permute(0,2,1) # way, d, shot*resolution

        if Woodbury:
            # correspond to Equation 10 in the paper
            
            sts = st.matmul(support) # way, d, d
            m_inv = (sts+torch.eye(sts.size(-1)).to(sts.device).unsqueeze(0).mul(lam)).inverse() # way, d, d
            hat = m_inv.matmul(sts) # way, d, d
        
        else:
            # correspond to Equation 8 in the paper
            
            sst = support.matmul(st) # way, shot*resolution, shot*resolution
            m_inv = (sst+torch.eye(sst.size(-1)).to(sst.device).unsqueeze(0).mul(lam)).inverse() # way, shot*resolution, shot*resolutionsf 
            hat = st.matmul(m_inv).matmul(support) # way, d, d

        Q_bar = query.matmul(hat).mul(rho) # way, way*query_shot*resolution, d

        dist = (Q_bar-query.unsqueeze(0)).pow(2).sum(2).permute(1,0) # way*query_shot*resolution, way
        
        return dist

    
    def get_neg_l2_dist(self,support_inp, query_inp, batch):
        
        resolution = self.resolution
        d = self.d
        alpha = self.r[0]
        beta = self.r[1]
        
        support_feature_map = self.get_feature_map(support_inp)
#         print('Input Shape: ', support_inp.shape)
#         print('Suport Feature Map: ', support_feature_map.shape)
        query_feature_map = self.get_feature_map(query_inp)
#         print('Query Feature Map: ', query_feature_map.shape)
        support = support_feature_map.view(194, resolution , d)
        query = query_feature_map.view(batch*resolution, d)
        recon_dist = self.get_recon_dist(query=query, support=support, alpha=alpha,beta=beta) # way*query_shot*resolution, way
#         print('Reconstructed Q: ', recon_dist.shape)
        neg_l2_dist = recon_dist.neg().view(batch,resolution,194).mean(1) # way*query_shot, way
        return neg_l2_dist, support


    def meta_test(self,support_inp, query_inp, batch):
        neg_l2_dist = self.get_neg_l2_dist(support_inp=support_inp, query_inp = query_inp, batch=batch)
        max_values,max_index = torch.max(neg_l2_dist[0],1)
        return max_values, max_index, neg_l2_dist


    def forward(self,inp):

        neg_l2_dist, support = self.get_neg_l2_dist(inp=inp,
                                                    way=self.way,
                                                    shot=self.shots[0],
                                                    query_shot=self.shots[1])
            
        logits = neg_l2_dist*self.scale
        log_prediction = F.log_softmax(logits,dim=1)

        return log_prediction, support

In [80]:
def get_device_map(gpu):
    cuda = lambda x: 'cuda:%d'%x
    temp = {}
    for i in range(4):
        temp[cuda(i)]=cuda(gpu)
    return temp

In [81]:
support_data = pd.read_csv('../input/snap-retail-data/final_data/support.csv')
support_data.drop_duplicates('class', inplace=True)
valid_query_data = pd.read_csv('../input/snap-retail-data/final_data/valid_query.csv')
test_query_data = pd.read_csv('../input/snap-retail-data/final_data/test_query.csv')
test_query_data.drop(test_query_data[test_query_data['image_id'] == '.DS_Store'].index, inplace=True)



data = pd.concat([support_data, valid_query_data, test_query_data], ignore_index=True)
class_codes = {}
for idx,cl in enumerate(data['class'].unique()):
    class_codes[str(cl)] = idx
    
for idx in data['class'].index:
    data.loc[idx,'class_code'] = class_codes[str(data.loc[idx,'class'])]
    
data['class_code'] = data['class_code'].astype(int)

BS = 194
BS_test = 1
support_df = data[data['type'] == 'support'].reset_index()
valid_query_df = data[data['type'] == 'valid'].reset_index()
test_query_df = data[data['type'] == 'test'].reset_index()
final_test_data_20 = pd.DataFrame()
for cl_,val in zip(test_query_df['class'].value_counts().index,test_query_df['class'].value_counts().values):
    if val >= 20:
        df = test_query_df[test_query_df['class'] == cl_][:20]
        final_test_data_20 = pd.concat([final_test_data_20, df], ignore_index = True)
dataset = CustomDataset_Support(support_df)
support_data_loader = torch.utils.data.DataLoader(dataset, batch_size=194)
valid_query_dataset = CustomDataset_Query(valid_query_df)
valid_query_data_loader = torch.utils.data.DataLoader(valid_query_dataset, batch_size=BS)
test_query_dataset = CustomDataset_Query(final_test_data_20, is_trained_class = False)
test_query_data_loader = torch.utils.data.DataLoader(test_query_dataset, batch_size=BS_test)

model_path = '../input/2-shot-learning/best_model_ResNet-12-sgd-lr_1e-01-gamma_1e-01-epoch_100-stage_2-decay_5e-04-way_10.pth'
gpu = 0
torch.cuda.set_device(gpu)

model = FRN()
model.cuda()
model.load_state_dict(torch.load(model_path,map_location=get_device_map(gpu)),strict=True)
model.eval()
with torch.no_grad():
    c = 0
#     for data_load, batch, df in zip([valid_query_data_loader, test_query_data_loader],[BS,BS_test],[valid_query_df, test_query_df]):
    for data_load, batch, df in zip([test_query_data_loader],[BS_test],[test_query_df]):
        for i, (inp,_) in tqdm(enumerate(support_data_loader)):
            inp = inp.cuda()
            support_inp = inp
        true_label = []
        pred_label = []
        max_value = []
        for i, (inp,target) in tqdm(enumerate(data_load)):
            inp = inp.cuda()
            target = target.cuda()
            max_values, max_index, neg_l2_dist = model.meta_test(support_inp, inp, batch)
            true_label.extend(list(target.cpu().numpy()))
            pred_label.extend(list(max_index.cpu().numpy()))
            max_value.extend(list(max_values.cpu().numpy()))
            

1it [00:00,  2.34it/s]
3140it [24:48,  2.11it/s]


In [82]:
df = pd.DataFrame()
df['Pred Score'] = max_value
df['True_Label'] = np.array(true_label) 
df['Predicted_Label'] = np.array(pred_label)
final_test_data_20 = pd.concat([final_test_data_20, df], axis=1)
final_test_data_20.shape

(3140, 8)

## Images Less than 20

In [83]:
support_data = pd.read_csv('../input/snap-retail-data/final_data/support.csv')
support_data.drop_duplicates('class', inplace=True)
test_query_data = pd.read_csv('../input/snap-retail-data/final_data/test_query.csv')
test_query_data.drop(test_query_data[test_query_data['image_id'] == '.DS_Store'].index, inplace=True)

data = pd.concat([support_data, valid_query_data, test_query_data], ignore_index=True)
class_codes = {}
for idx,cl in enumerate(data['class'].unique()):
    class_codes[str(cl)] = idx
    
for idx in data['class'].index:
    data.loc[idx,'class_code'] = class_codes[str(data.loc[idx,'class'])]
    
data['class_code'] = data['class_code'].astype(int)

BS = 194
BS_test = 1
support_df = data[data['type'] == 'support'].reset_index()
test_query_df = data[data['type'] == 'test'].reset_index()
final_test_data_20_5 = pd.DataFrame()
for cl_,val in zip(test_query_df['class'].value_counts().index,test_query_df['class'].value_counts().values):
    if val < 20 and val>=5:
        df = test_query_df[test_query_df['class'] == cl_]
        final_test_data_20_5 = pd.concat([final_test_data_20_5, df], ignore_index = True)
dataset = CustomDataset_Support(support_df)
support_data_loader = torch.utils.data.DataLoader(dataset, batch_size=194)
valid_query_dataset = CustomDataset_Query(valid_query_df)
valid_query_data_loader = torch.utils.data.DataLoader(valid_query_dataset, batch_size=BS)
test_query_dataset = CustomDataset_Query(final_test_data_20_5, is_trained_class = False)
test_query_data_loader = torch.utils.data.DataLoader(test_query_dataset, batch_size=BS_test)

model_path = '../input/2-shot-learning/best_model_ResNet-12-sgd-lr_1e-01-gamma_1e-01-epoch_100-stage_2-decay_5e-04-way_10.pth'
gpu = 0
torch.cuda.set_device(gpu)

model = FRN()
model.cuda()
model.load_state_dict(torch.load(model_path,map_location=get_device_map(gpu)),strict=True)
model.eval()
with torch.no_grad():
    c = 0
#     for data_load, batch, df in zip([valid_query_data_loader, test_query_data_loader],[BS,BS_test],[valid_query_df, test_query_df]):
    for data_load, batch, df in zip([test_query_data_loader],[BS_test],[test_query_df]):
        for i, (inp,_) in tqdm(enumerate(support_data_loader)):
            inp = inp.cuda()
            support_inp = inp
        true_label = []
        pred_label = []
        max_value = []
        for i, (inp,target) in tqdm(enumerate(data_load)):
            inp = inp.cuda()
            target = target.cuda()
            max_values, max_index, neg_l2_dist = model.meta_test(support_inp, inp, batch)
            true_label.extend(list(target.cpu().numpy()))
            pred_label.extend(list(max_index.cpu().numpy()))
            max_value.extend(list(max_values.cpu().numpy()))
            

1it [00:00,  2.34it/s]
334it [02:38,  2.11it/s]


In [110]:
df = pd.DataFrame()
df['Pred Score'] = max_value
df['True_Label'] = np.array(true_label) 
df['Predicted_Label'] = np.array(pred_label)
final_test_data_20_5 = pd.concat([final_test_data_20_5, df], axis=1)
final_test_data_20_5.shape

(334, 8)

In [115]:
final_test_data_20_5['accuracy'] = 1*(final_test_data_20_5['Predicted_Label'] == final_test_data_20_5['True_Label'])

In [116]:
final_test_data_20_5.head()

,index,image_id,class,type,class_code,Pred Score,True_Label,Predicted_Label,accuracy
0,1162,walmart-neighborhood-market-5657_16514983_Q02-...,999999981529,test,7,-0.301439,7,7,1
1,2126,walmart-neighborhood-market-5855_16497595_Q02-...,999999981529,test,7,-0.536676,7,7,1
2,2514,walmart-neighborhood-market-5613_16495482_Q02-...,999999981529,test,7,-0.345235,7,7,1
3,2598,walmart-neighborhood-market-5657_16514983_Q02-...,999999981529,test,7,-0.378837,7,7,1
4,2897,walmart-supercenter-1521_16510774_Q02-002_zIK9...,999999981529,test,7,-0.521002,7,7,1


In [118]:
accu =  []
for cl_ in final_test_data_20_5['class']:
    d = final_test_data_20_5[final_test_data_20_5['class'] == cl_]
    accu.append(d['accuracy'].sum()/len(d))
accu = np.mean(accu)
accu

0.811377245508982

In [119]:
final_data_df = pd.concat([final_test_data_20, final_test_data_20_5])
final_data_df.head()

,index,image_id,class,type,class_code,Pred Score,True_Label,Predicted_Label,accuracy
0,388,walmart-supercenter-1238_16505805_Q02-004_pLiM...,999999981516,test,155,-0.317165,155,155,NaN
1,513,walmart-supercenter-1080_16507175_Q02-005_7MpQ...,999999981516,test,155,-0.323810,155,155,NaN
2,551,walmart-supercenter-1117_16514977_Q02-001_JD5x...,999999981516,test,155,-0.364363,155,28,NaN
3,656,walmart-supercenter-147_16507331_Q02-003_2Zzer...,999999981516,test,155,-0.398736,155,155,NaN
4,673,walmart-supercenter-1433_16500483_Q02-005_ZULm...,999999981516,test,155,-0.282852,155,28,NaN


In [121]:
final_data_df['accuracy'] =  1*(final_data_df['Predicted_Label'] == final_data_df['True_Label'])

In [124]:
overall_accu =  []
for cl_ in final_data_df['class']:
    d = final_data_df[final_data_df['class'] == cl_]
    overall_accu.append((d['accuracy'].sum()/len(d)))
# overall_accu = overall_accu/final_data_df['class'].nunique()
overall_accu = np.mean(overall_accu)
overall_accu

0.696027633851468